In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader(file_path)
documents = loader.load()

In [ ]:
text_without_newlines
for document in documents:
    text = document.page_content
    text_without_newlines = re.sub(r'\n+', ' ', text)


text_without_newlines

In [ ]:
import requests

# Ollama server details
OLLAMA_URL = "http://127.0.0.1:12345"

def get_nomic_embedding(text):
    payload = {
        "model": "nomic-embed-text",
        "prompt": text,
        "embed": True
    }
    response = requests.post(f"{OLLAMA_URL}/api/embeddings", json=payload)
    if response.status_code == 200:
        return response.json().get("embedding")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Test with a sample text
text = "The quick brown fox jumps over the lazy."
embedding = get_nomic_embedding(text)

if embedding:
    print("Embedding Generated:")
    print(embedding)


In [ ]:
from langchain.llms import Ollama

# Connect to the Ollama server
ollama_llm = Ollama(
    model="llama3",
    base_url="http://127.0.0.1:12345"
)

# Test with a prompt
prompt = "who is samrat abdul jalil"
response = ollama_llm(prompt)
print(response)

In [ ]:
#with this code add croma db as vector database create collection using file path name . make chunk ,overlap using langchain. when i qury in chroma db it will give me 3 chunck .using this data send it to ollama for genarating text according to query full local rag system . now give code sell wise . like pdf loading , chroma query then ollama text genaration . i will use it in jupiter notebook .  

In [11]:
# Step 1: PDF Loading
import re
from langchain.document_loaders import UnstructuredPDFLoader

file_path = "machine learning.pdf"
loader = UnstructuredPDFLoader(file_path)
documents = loader.load()

texts = []
for document in documents:
    text = document.page_content
    text_without_newlines = re.sub(r'\n+', ' ', text)
    texts.append(text_without_newlines)

# Step 2: Chroma Setup with `nomic-embed-text`
import chromadb
import requests

OLLAMA_URL = "http://127.0.0.1:12345"

def get_nomic_embedding(text):
    payload = {
        "model": "nomic-embed-text",
        "prompt": text,
        "embed": True
    }
    response = requests.post(f"{OLLAMA_URL}/api/embeddings", json=payload)
    if response.status_code == 200:
        return response.json().get("embedding")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

client = chromadb.Client()
collection_name = "machin22e"
collection = client.create_collection(collection_name)

# Generate embeddings for each text chunk and add to Chroma collection
embeddings = [get_nomic_embedding(text) for text in texts]
collection.add(texts, embeddings=embeddings)

# Step 3: Chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Step 4: Querying Chroma DB
def query_chroma_db(query, collection):
    query_embedding = get_nomic_embedding(query)
    results = collection.query(query_embedding, n_results=1)
    print(results["documents"])
    return results["documents"]

query = "What is the topic of the document?"
relevant_chunks = query_chroma_db(query, collection)

# Step 5: Text Generation with Ollama
def generate_ollama_response(prompt):
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "max_tokens": 200
    }
    response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

full_input = " ".join(relevant_chunks)
generated_text = generate_ollama_response(full_input)

print("Generated Text:")
print(generated_text)


UniqueConstraintError: Collection machin2e already exists

In [13]:
# Step 1: PDF Loading
import re
from langchain.document_loaders import UnstructuredPDFLoader

file_path = "machine learning.pdf"
loader = UnstructuredPDFLoader(file_path)
documents = loader.load()

texts = []
for document in documents:
    text = document.page_content
    text_without_newlines = re.sub(r'\n+', ' ', text)
    texts.append(text_without_newlines)

# Step 2: Chroma Setup with `nomic-embed-text`
import chromadb
import requests

OLLAMA_URL = "http://127.0.0.1:12345"

def get_nomic_embedding(text):
    payload = {
        "model": "nomic-embed-text",
        "prompt": text,
        "embed": True
    }
    response = requests.post(f"{OLLAMA_URL}/api/embeddings", json=payload)
    if response.status_code == 200:
        print("Embedding successful for text: ", text[:30])  # Print part of the text to confirm embedding
        return response.json().get("embedding")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

client = chromadb.Client()
collection_name = "machin212e"
collection = client.create_collection(collection_name)

# Generate embeddings for each text chunk and add to Chroma collection
embeddings = []
for text in texts:
    embedding = get_nomic_embedding(text)
    if embedding:
        embeddings.append(embedding)

# Ensure that embeddings are created successfully before adding them to Chroma
if embeddings:
    print("Embedding generation successful. Now storing in Chroma...")
    collection.add(texts, embeddings=embeddings)
    print("Successfully stored in Chroma.")
else:
    print("No embeddings were generated.")

# Step 3: Chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Print the first few chunks to verify chunking
print("Chunks created successfully:")
for chunk in chunks[:3]:
    print(chunk.page_content[:100])  # Print the first 100 characters of each chunk

# Step 4: Querying Chroma DB
def query_chroma_db(query, collection):
    query_embedding = get_nomic_embedding(query)
    results = collection.query(query_embedding, n_results=1)
    print("Querying successful. Retrieved results:")
    print(results["documents"])  # Print the retrieved documents
    return results["documents"]

query = "What is the topic of the document?"
relevant_chunks = query_chroma_db(query, collection)

# Step 5: Text Generation with Ollama
def generate_ollama_response(prompt):
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "max_tokens": 200
    }
    response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload)
    if response.status_code == 200:
        print("Generated text from Ollama successfully.")
        return response.json().get("generated_text")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Flatten the list of relevant chunks before passing to the model
if relevant_chunks:
    full_input = " ".join([str(chunk) for chunk in relevant_chunks])
    generated_text = generate_ollama_response(full_input)

    print("Generated Text:")
    print(generated_text)
else:
    print("No relevant chunks found for generation.")


Embedding successful for text:  SAMRAT ABDUL JALIL AI and Back
Embedding generation successful. Now storing in Chroma...
Successfully stored in Chroma.
Chunks created successfully:
SAMRAT ABDUL JALIL AI and Backend Enthusiast

SKILLS

Back-End: Python, C/C++,Java ,Php, FastAPI, Ex
WORK EXPERIENCE

Undergraduate Thesis: GradeVision (in collaboration with Edusoftbd Ltd) ● Developed
Tech Stack : Python, FastAPI, Flutter, MySQL .

DL Sprint 2.0: Bengali Document Layout Analysis ● Fi
Embedding successful for text:  What is the topic of the docum
Querying successful. Retrieved results:
[[None]]
Generated text from Ollama successfully.


JSONDecodeError: Extra data: line 2 column 1 (char 92)

In [16]:
# Step 1: PDF Loading
import re
from langchain.document_loaders import UnstructuredPDFLoader

file_path = "machine learning.pdf"
loader = UnstructuredPDFLoader(file_path)
documents = loader.load()

texts = []
for document in documents:
    text = document.page_content
    text_without_newlines = re.sub(r'\n+', ' ', text)
    texts.append(text_without_newlines)

# Step 2: Chroma Setup with `nomic-embed-text`
import chromadb
import requests

OLLAMA_URL = "http://127.0.0.1:12345"

def get_nomic_embedding(text):
    payload = {
        "model": "nomic-embed-text",
        "prompt": text,
        "embed": True
    }
    response = requests.post(f"{OLLAMA_URL}/api/embeddings", json=payload)
    if response.status_code == 200:
        embedding = response.json().get("embedding")
        print(f"Embedding successful for text: {text[:30]}...")  # Print part of the text to confirm embedding
        print(f"Embedding vector: {embedding[:10]}...")  # Print the first 10 values of the embedding vector
        return embedding
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

client = chromadb.Client()
collection_name = "machin21142e"
collection = client.create_collection(collection_name)

# Generate embeddings for each text chunk and add to Chroma collection
embeddings = []
for text in texts:
    embedding = get_nomic_embedding(text)
    if embedding:
        embeddings.append(embedding)

# Ensure that embeddings are created successfully before adding them to Chroma
if embeddings:
    print("Embedding generation successful. Now storing in Chroma...")
    collection.add(texts, embeddings=embeddings)
    print("Successfully stored in Chroma.")
else:
    print("No embeddings were generated.")

# Step 3: Chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

# Print the first few chunks to verify chunking
print("Chunks created successfully:")
for chunk in chunks[:3]:
    print(chunk.page_content[:100])  # Print the first 100 characters of each chunk

# Step 4: Querying Chroma DB
def query_chroma_db(query, collection):
    query_embedding = get_nomic_embedding(query)
    if query_embedding is None:
        print("Failed to generate embedding for the query.")
        return []
    print(f"Query embedding generated: {query_embedding[:10]}...")  # Print first 10 values of query embedding
    results = collection.query(query_embedding, n_results=1)
    
    # Debug: Print the retrieved documents and their embeddings
    print("Querying successful. Retrieved results:")
    print(results["documents"])  # Print the retrieved documents
    print("Retrieved embeddings for the documents:")
    for doc in results["documents"]:
        print(f"Document: {doc}")
        # You can check the embedding or content in the document as needed
    return results["documents"]

query = "SAMRAT ABDUL JALIL?"
relevant_chunks = query_chroma_db(query, collection)

# Step 5: Text Generation with Ollama
def generate_ollama_response(prompt):
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "max_tokens": 200
    }
    response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload)
    if response.status_code == 200:
        print("Generated text from Ollama successfully.")
        return response.json().get("generated_text")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Flatten the list of relevant chunks before passing to the model
if relevant_chunks:
    full_input = " ".join([str(chunk) for chunk in relevant_chunks])
    generated_text = generate_ollama_response(full_input)

    print("Generated Text:")
    print(generated_text)
else:
    print("No relevant chunks found for generation.")


Embedding successful for text: SAMRAT ABDUL JALIL AI and Back...
Embedding vector: [-0.6538950204849243, 0.2558712065219879, -2.2708866596221924, -0.2998715341091156, 0.6298578977584839, -0.6392511129379272, 0.7918146848678589, -0.08320767432451248, 0.6520728468894958, -0.5487140417098999]...
Embedding generation successful. Now storing in Chroma...
Successfully stored in Chroma.
Chunks created successfully:
SAMRAT ABDUL JALIL AI and Backend Enthusiast

SKILLS

Back-End: Python, C/C++,Java ,Php, FastAPI, Ex
WORK EXPERIENCE

Undergraduate Thesis: GradeVision (in collaboration with Edusoftbd Ltd) ● Developed
Tech Stack : Python, FastAPI, Flutter, MySQL .

DL Sprint 2.0: Bengali Document Layout Analysis ● Fi
Embedding successful for text: SAMRAT ABDUL JALIL?...
Embedding vector: [0.38792097568511963, -0.1647554337978363, -3.3390893936157227, 0.1795051395893097, 0.08914053440093994, 0.7341902852058411, -0.6030028462409973, 0.40323227643966675, -0.11605999618768692, 0.5501057505607605]...
Q

JSONDecodeError: Extra data: line 2 column 1 (char 92)